### Imports

In [3]:
!pip install azure-cognitiveservices-speech
!pip install trafilatura
!pip install summa
!pip install git+https://github.com/smirnov-am/pytopicrank.git#egg=pytopicrank
!pip install git+https://github.com/LIAAD/yake
!pip install rake_nltk
!pip install numpy==1.17.0
!pip install scipy
!pip install scikit-learn

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-8o48xc6p
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-8o48xc6p
  Created wheel for yake: filename=yake-0.4.5-py2.py3-none-any.whl size=60006 sha256=0582463519c009120ee472e756bf1d3259c83b2e3c9f2fd0be6f81f7028bebdd
  Stored in directory: /tmp/pip-ephem-wheel-cache-373868gg/wheels/3c/18/3d/c22231fb773c623a51f73209e3356084f02e7b72d1032806b8
Successfully built yake


In [4]:
import time
import nltk
import trafilatura
import re
import numpy as np
import pandas as pd
import requests
from nltk.corpus import stopwords
from rake_nltk import Rake, Metric
from pytopicrank import TopicRank
from yake import KeywordExtractor
from bs4 import BeautifulSoup
import azure.cognitiveservices.speech as speechsdk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Utilities

In [49]:
def speech_recognize_continuous_from_file(filename):
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription="4d209e4fd5234011a8df81e44166d116",
                                           region="canadacentral")
    audio_config = speechsdk.audio.AudioConfig(filename=filename)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        """callback that stops continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    all_results = []
    def handle_final_result(evt):
        all_results.append(evt.result.text)

    speech_recognizer.recognized.connect(handle_final_result)
    # Connect callbacks to the events fired by the speech recognizer
#     speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
#     speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
#     speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
#     speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
#     speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)
    print('Connecting Session...')

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    print('Recognizing Speech...')
    while not done:
        time.sleep(.5)
    return all_results

def wikiCorpusSearch(wordlist, top_N = 10):
  N_wrds = []
  _count = 0

  for wrd in wordlist:
    wrd = wrd.replace(' ', '_')
    _PATH = "https://en.wikipedia.org/wiki/{}".format(wrd)
    # print(_PATH)

    if requests.get(_PATH).status_code == 200:
      _count += 1
      N_wrds.append((wrd,_PATH))
      if _count == top_N:
        break
    else:
      pass
  
  return N_wrds

def wikiSearch(wrd):
  wrd = wrd.replace(' ', '_')
  _URL = "https://en.wikipedia.org/wiki/{}".format(wrd)
  # print(_PATH)

  ### GET RESPONSE
  response = requests.get(_URL)
  # print(_URL)
  _soup = BeautifulSoup(response.content, 'html.parser')
  _table = _soup.find_all(id="mw-content-text")

  _paragraph = []
  for element in _table:
    for content in element.find_all('p'):
      if len(content.get_text()) == 1:
        pass
      else:
        if ('may also refer to:' not in content.get_text()) & ('may refer to:' not in content.get_text()):
            _paragraph.append(content.get_text())

  return '\n'.join(_paragraph[0:5])

# paragraphs = wikiSearch(L)

def format_text(original, keywords_list):
    with open('Notes.txt', 'w+') as note_file:
        note_file.write('ORIGINAL CONTEXT:\n')
        note_file.write(original + '\n')
        note_file.write('\n')

        note_file.write('Keywords:\n')
        for keywords in keywords_list:
            intro_paragraph = wikiSearch(keywords)            
            if len(intro_paragraph) == 0:
                pass
            else:
                note_file.write(keywords + ':\n')
                note_file.write(intro_paragraph)
                note_file.write('\n' + '-'*20 + '\n')

### MAIN

In [50]:
def main():
    results = speech_recognize_continuous_from_file(filename)
    text = ' '.join(results)
    print('Recognized Text:\n' + text)
    top_ = int(int(len(text.split())) / 5)
    
    ### GET RAKE Keywords
    r = Rake(min_length=2, 
             max_length=4,
             stopwords=stopwords.words('english'))
    r.extract_keywords_from_text(text)
    kwrds = r.get_ranked_phrases()
    keywords = wikiCorpusSearch(kwrds, top_N = top_)
    
    ### GET KEYWORDS BY TopicRank
    tr = TopicRank(text)
    tp_kwrds = tr.get_top_n(n=top_, extract_strategy='first')
    tp_keywords = wikiCorpusSearch(tp_kwrds, top_N = top_)
    
    ### GET KEYWORDS BY YAKE!
    kw_extractor = KeywordExtractor(lan="en", 
                                n=4,
                                top= top_,
                                windowsSize = 10)
    yake_keywords = kw_extractor.extract_keywords(text)
    yake_keywords = pd.DataFrame(yake_keywords, columns= ['words', 'score']).sort_values('score', ascending=False)
    yake_keywords = wikiCorpusSearch(list(yake_keywords['words']), top_N = top_)
    
    ### Need to simplify a bit...
    L = []
    prev_L = []
    for l in [keywords, tp_keywords, yake_keywords]:
        current = [wrd for (wrd,url) in l if wrd not in prev_L]
        L.append(current)
        prev_L = [wrd for sublist in L for wrd in sublist]
    L = [wrd for sublist in L for wrd in sublist]

    ### GENERATE NOTES
    format_text(text, L)
    
    return 0

In [53]:
filename = 'sample2.wav'

main()

Connecting Session...
Recognizing Speech...
CLOSING on SpeechRecognitionCanceledEventArgs(session_id=1ffd271a07804bbf8366123bc04df4aa, result=SpeechRecognitionResult(result_id=9135827f434a4d12914d242adb6d5b43, text="", reason=ResultReason.Canceled))
CLOSING on SessionEventArgs(session_id=1ffd271a07804bbf8366123bc04df4aa)


2021-04-01 23:36:13,173: DEBUG: Starting new HTTPS connection (1): en.wikipedia.org:443
2021-04-01 23:36:13,338: DEBUG: https://en.wikipedia.org:443 "GET /wiki/rest_api_alternative_speech HTTP/1.1" 301 0


Recognized Text:
The Speech SDK exposes many features from the speech service, but not all of them. The capabilities of the Speech SDK are often associated with scenarios. The Speech SDK is ideal for both real time and non real time scenarios using local devices files, Azure blob storage and even input and output streams. When a scenario is not achievable with the speech SDK, look for a REST API alternative speech to text, also known as speech recognition, transcribes audio streams to text that your applications, tools, or devices can consume or display. Use speech to text with language understanding Lewis to derive user intents from transcribed speech and act on voice commands. You speech translation to translate speech input to a different language with a single call. For more information, see speech to text basics.


2021-04-01 23:36:13,368: DEBUG: https://en.wikipedia.org:443 "GET /wiki/Rest_api_alternative_speech HTTP/1.1" 404 6711
2021-04-01 23:36:13,375: DEBUG: Starting new HTTPS connection (1): en.wikipedia.org:443
2021-04-01 23:36:13,463: DEBUG: https://en.wikipedia.org:443 "GET /wiki/derive_user_intents HTTP/1.1" 301 0
2021-04-01 23:36:13,494: DEBUG: https://en.wikipedia.org:443 "GET /wiki/Derive_user_intents HTTP/1.1" 404 6694
2021-04-01 23:36:13,500: DEBUG: Starting new HTTPS connection (1): en.wikipedia.org:443
2021-04-01 23:36:13,587: DEBUG: https://en.wikipedia.org:443 "GET /wiki/azure_blob_storage HTTP/1.1" 301 0
2021-04-01 23:36:13,617: DEBUG: https://en.wikipedia.org:443 "GET /wiki/Azure_blob_storage HTTP/1.1" 404 6695
2021-04-01 23:36:13,622: DEBUG: Starting new HTTPS connection (1): en.wikipedia.org:443
2021-04-01 23:36:13,710: DEBUG: https://en.wikipedia.org:443 "GET /wiki/transcribes_audio_streams HTTP/1.1" 301 0
2021-04-01 23:36:13,742: DEBUG: https://en.wikipedia.org:443 "GET /

0